In [3]:
import os
import random
import shutil

# ---------------------------
# 1. Split the Data into Train/Val
# ---------------------------
# Set a seed for reproducibility
random.seed(42)

# Define the path to the directory containing the patient folders.
train_dir = './dataset/train'  # Adjust this path if necessary

# List all folders that start with "patient" and are directories.
patient_dirs = [d for d in os.listdir(train_dir) 
                if d.startswith("patient") and os.path.isdir(os.path.join(train_dir, d))]
patient_dirs.sort()  # Ensure a consistent order before shuffling

print("Total number of patient folders:", len(patient_dirs))

# Shuffle the patient folders to randomize the split
random.shuffle(patient_dirs)

# Compute the split index (80% for training, 20% for validation)
split_index = int(0.8 * len(patient_dirs))
train_patients = patient_dirs[:split_index]
val_patients = patient_dirs[split_index:]

print("Number of training patients:", len(train_patients))
print("Number of validation patients:", len(val_patients))

# Optionally, display the lists
print("\nTraining patients:")
print(train_patients)

print("\nValidation patients:")
print(val_patients)

# Optional: Copy the split patient folders to separate directories.
# This step physically separates the training and validation data.
train_out_dir = './dataset/train_split'
val_out_dir = './dataset/val_split'

os.makedirs(train_out_dir, exist_ok=True)
os.makedirs(val_out_dir, exist_ok=True)

# Copy training patient folders
for patient in train_patients:
    src = os.path.join(train_dir, patient)
    dst = os.path.join(train_out_dir, patient)
    shutil.copytree(src, dst, dirs_exist_ok=True)

# Copy validation patient folders
for patient in val_patients:
    src = os.path.join(train_dir, patient)
    dst = os.path.join(val_out_dir, patient)
    shutil.copytree(src, dst, dirs_exist_ok=True)

print("\nData splitting complete.")
print(f"Training data copied to: {train_out_dir}")
print(f"Validation data copied to: {val_out_dir}")

# ---------------------------
# 2. Resample the Images to 1.25 x 1.25 mm In-Plane Resolution
# ---------------------------
import SimpleITK as sitk

# Set the target in-plane resolution (width and height) in mm.
target_resolution = [1.25, 1.25]

def resample_image(image, target_resolution, interpolator=sitk.sitkLinear):
    """
    Resample a 2D or 3D image so that its first two dimensions (x and y) have the specified resolution.
    The third dimension (if present) is left unchanged.
    """
    original_spacing = image.GetSpacing()   # e.g., (sx, sy, [sz])
    original_size = image.GetSize()           # e.g., (nx, ny, [nz])
    
    # Compute the new size for x and y
    new_size = list(original_size)
    new_size[0] = int(round(original_size[0] * original_spacing[0] / target_resolution[0]))
    new_size[1] = int(round(original_size[1] * original_spacing[1] / target_resolution[1]))
    
    # Create new spacing: update x and y; leave others unchanged.
    new_spacing = list(original_spacing)
    new_spacing[0] = target_resolution[0]
    new_spacing[1] = target_resolution[1]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(interpolator)
    
    return resampler.Execute(image)

# Process both train_split and val_split directories.
base_dirs = [train_out_dir, val_out_dir]

for base_dir in base_dirs:
    if not os.path.exists(base_dir):
        print("Directory does not exist:", base_dir)
        continue
    
    print("Processing directory:", base_dir)
    for patient in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                # Process only files with "frame" in their name, excluding those with "4d".
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    
                    # Read the image using SimpleITK.
                    image = sitk.ReadImage(file_path)
                    spacing = image.GetSpacing()
                    
                    # Check if x and y spacing are already as desired (within a small tolerance).
                    if abs(spacing[0] - target_resolution[0]) < 1e-6 and abs(spacing[1] - target_resolution[1]) < 1e-6:
                        print("Skipping already resampled:", file_path)
                        continue
                    
                    print("Resampling:", file_path)
                    
                    # Use nearest neighbor interpolation for ground truth images to preserve labels.
                    if '_gt' in file:
                        interp = sitk.sitkNearestNeighbor
                    else:
                        interp = sitk.sitkLinear
                    
                    # Resample the image.
                    resampled_image = resample_image(image, target_resolution, interpolator=interp)
                    
                    # Overwrite the original file with the resampled image.
                    sitk.WriteImage(resampled_image, file_path)

print("Resampling for train_split and val_split complete.")

# ---------------------------
# 3. Intensity Normalization (Z-score)
# ---------------------------
import numpy as np

def zscore_normalization(image):
    """
    Normalize the image using Z-score normalization:
      (image - mean) / std
    """
    # Convert the image to a NumPy array.
    arr = sitk.GetArrayFromImage(image)
    mean_val = np.mean(arr)
    std_val = np.std(arr)
    
    # Prevent division by zero.
    if std_val == 0:
        std_val = 1
    
    normalized_arr = (arr - mean_val) / std_val
    
    # Convert the normalized array back to a SimpleITK image and preserve metadata.
    normalized_image = sitk.GetImageFromArray(normalized_arr)
    normalized_image.CopyInformation(image)
    return normalized_image

# Process both train_split and val_split directories.
base_dirs = [train_out_dir, val_out_dir]

for base_dir in base_dirs:
    if not os.path.exists(base_dir):
        print("Directory does not exist:", base_dir)
        continue

    print("Processing directory for normalization:", base_dir)
    for patient in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                # Process only files with "frame" in their name, excluding "4d" files.
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    # Skip ground truth images.
                    if '_gt' not in file:
                        file_path = os.path.join(patient_path, file)
                        print("Normalizing:", file_path)
                        
                        # Read the image.
                        image = sitk.ReadImage(file_path)
                        
                        # Normalize the image.
                        normalized_image = zscore_normalization(image)
                        
                        # Overwrite the original file with the normalized image.
                        sitk.WriteImage(normalized_image, file_path)

print("Intensity normalization complete.")

# ---------------------------
# 4. Resize Images to 352 x 352 Pixels for 2D U-Net Input
# ---------------------------
# Desired output size for the network input.
target_size = (352, 352)

def resize_image_to_fixed_size(image, output_size, interpolator=sitk.sitkLinear):
    """
    Resize a 2D (or 3D with a single slice dimension) image to a fixed output size.
    For 3D images, only the first two dimensions (x, y) are resized and the slice dimension is kept unchanged.
    """
    dim = image.GetDimension()
    original_size = list(image.GetSize())
    original_spacing = list(image.GetSpacing())
    
    if dim == 2:
        # Compute new spacing for a 2D image.
        new_size = list(output_size)
        new_spacing = [ (original_size[i] * original_spacing[i]) / output_size[i] for i in range(2)]
    elif dim == 3:
        # For a 3D image, assume the third dimension is the slice dimension.
        new_size = list(output_size) + [original_size[2]]
        new_spacing_xy = [ (original_size[i] * original_spacing[i]) / output_size[i] for i in range(2)]
        new_spacing = new_spacing_xy + [original_spacing[2]]
    else:
        raise ValueError("Image dimension not supported.")
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(new_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(interpolator)
    
    return resampler.Execute(image)

# Process directories: here we assume val_split and test_split need resizing.
base_dirs = ['./dataset/val_split', './dataset/test_split']

for base_dir in base_dirs:
    if not os.path.exists(base_dir):
        print("Directory does not exist:", base_dir)
        continue
    
    print("Processing directory for resizing:", base_dir)
    for patient in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                # Process only files with "frame" in the name and ignore any with "4d".
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    print("Resizing:", file_path)
                    
                    # Read the image.
                    image = sitk.ReadImage(file_path)
                    
                    # Use nearest neighbor for ground truth images, linear for others.
                    if '_gt' in file:
                        interp = sitk.sitkNearestNeighbor
                    else:
                        interp = sitk.sitkLinear
                    
                    # Resize the image to 352 x 352 pixels.
                    resized_image = resize_image_to_fixed_size(image, target_size, interpolator=interp)
                    
                    # Overwrite the original file with the resized image.
                    sitk.WriteImage(resized_image, file_path)

print("Resizing complete for val_split and test_split.")

# ---------------------------
# 5. Extract 2D Slices from Volumes for 2D U-Net Training
# ---------------------------
# Here we extract individual 2D slices from each preprocessed volume (both frame and ground truth)
# and save them in new directories (train_2d and val_2d). These slices will serve as individual training samples.

def extract_slices(volume_image):
    """
    Extract 2D slices from a 3D image along the z-dimension using SimpleITK's ExtractImageFilter.
    Returns a list of 2D SimpleITK images.
    """
    # Get the full 3D image size (order: [width, height, depth])
    size = list(volume_image.GetSize())
    slices = []
    extractor = sitk.ExtractImageFilter()
    
    # Iterate over the z-dimension
    for z in range(size[2]):
        # Set extraction index: start at x=0, y=0, and at the current z slice.
        extraction_index = [0, 0, z]
        # Set extraction size: full width and height, but only one slice in z.
        extraction_size = [size[0], size[1], 0]  # 0 means "extract one slice" along that dimension.
        
        extractor.SetSize(extraction_size)
        extractor.SetIndex(extraction_index)
        slice_img = extractor.Execute(volume_image)
        slices.append(slice_img)
    return slices

# Create output directories for extracted 2D slices.
train_2d_dir = './dataset/train_2d'
val_2d_dir = './dataset/val_2d'
os.makedirs(train_2d_dir, exist_ok=True)
os.makedirs(val_2d_dir, exist_ok=True)

# Process both train_split and val_split to extract 2D slices.
for split_dir, output_dir in zip([train_out_dir, val_out_dir], [train_2d_dir, val_2d_dir]):
    print(f"Extracting 2D slices from {split_dir} ...")
    for patient in os.listdir(split_dir):
        patient_path = os.path.join(split_dir, patient)
        if os.path.isdir(patient_path):
            # Create corresponding patient folder in the output directory.
            out_patient_dir = os.path.join(output_dir, patient)
            os.makedirs(out_patient_dir, exist_ok=True)
            for file in os.listdir(patient_path):
                if 'frame' in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    print("Extracting slices from:", file_path)
                    image = sitk.ReadImage(file_path)
                    slices = extract_slices(image)
                    # Save each 2D slice as a separate file.
                    for i, slice_img in enumerate(slices):
                        base_name = os.path.splitext(file)[0]
                        new_filename = f"{base_name}_slice{i:02d}.nii"
                        new_file_path = os.path.join(out_patient_dir, new_filename)
                        sitk.WriteImage(slice_img, new_file_path)
                        print("Saved slice:", new_file_path)

print("Slice extraction complete.")


Total number of patient folders: 100
Number of training patients: 80
Number of validation patients: 20

Training patients:
['patient043', 'patient042', 'patient092', 'patient010', 'patient066', 'patient051', 'patient002', 'patient071', 'patient016', 'patient079', 'patient074', 'patient011', 'patient056', 'patient057', 'patient073', 'patient046', 'patient049', 'patient093', 'patient077', 'patient038', 'patient031', 'patient022', 'patient033', 'patient097', 'patient081', 'patient050', 'patient084', 'patient027', 'patient088', 'patient034', 'patient009', 'patient048', 'patient060', 'patient064', 'patient075', 'patient045', 'patient099', 'patient053', 'patient086', 'patient013', 'patient037', 'patient024', 'patient040', 'patient041', 'patient019', 'patient067', 'patient062', 'patient061', 'patient008', 'patient035', 'patient100', 'patient047', 'patient003', 'patient052', 'patient017', 'patient039', 'patient059', 'patient069', 'patient023', 'patient063', 'patient025', 'patient006', 'patient

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame01_gt.nii has unexpected scales in sform



Resampling: ./dataset/train_split/patient043/patient043_frame01.nii
Resampling: ./dataset/train_split/patient043/patient043_frame01_gt.nii


NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame07.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame07.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame07_gt.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/train_split/patient043/patient043_frame07_gt.nii has unexpected scales in sform



Resampling: ./dataset/train_split/patient043/patient043_frame07.nii
Resampling: ./dataset/train_split/patient043/patient043_frame07_gt.nii
Resampling: ./dataset/train_split/patient042/patient042_frame01.nii


...

Normalizing: ./dataset/train_split/patient043/patient043_frame07.nii
Normalizing: ./dataset/train_split/patient042/patient042_frame01.nii
Normalizing: ./dataset/train_split/patient042/patient042_frame16.nii
Normalizing: ./dataset/train_split/patient092/patient092_frame01.nii
Normalizing: ./dataset/train_split/patient092/patient092_frame06.nii
Normalizing: ./dataset/train_split/patient010/patient010_frame01.nii
Normalizing: ./dataset/train_split/patient010/patient010_frame13.nii
Normalizing: ./dataset/train_split/patient066/patient066_frame01.nii
Normalizing: ./dataset/train_split/patient066/patient066_frame11.nii
Normalizing: ./dataset/train_split/patient051/patient051_frame01.nii
Normalizing: ./dataset/train_split/patient051/patient051_frame11.nii
Normalizing: ./dataset/train_split/patient002/patient002_frame01.nii
Normalizing: ./dataset/train_split/patient002/patient002_frame12.nii
Normalizing: ./dataset/train_split/patient071/patient071_frame01.nii
Normalizing: ./dataset/train_split

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame09.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame09.nii has unexpected scales in sform



Normalizing: ./dataset/val_split/patient055/patient055_frame10.nii
Normalizing: ./dataset/val_split/patient076/patient076_frame01.nii
Normalizing: ./dataset/val_split/patient076/patient076_frame12.nii
Normalizing: ./dataset/val_split/patient012/patient012_frame01.nii
Normalizing: ./dataset/val_split/patient012/patient012_frame13.nii
Normalizing: ./dataset/val_split/patient070/patient070_frame01.nii
Normalizing: ./dataset/val_split/patient070/patient070_frame10.nii
Normalizing: ./dataset/val_split/patient087/patient087_frame01.nii
Normalizing: ./dataset/val_split/patient087/patient087_frame10.nii
Normalizing: ./dataset/val_split/patient014/patient014_frame01.nii
Normalizing: ./dataset/val_split/patient014/patient014_frame13.nii
Normalizing: ./dataset/val_split/patient018/patient018_frame01.nii
Normalizing: ./dataset/val_split/patient018/patient018_frame10.nii
Normalizing: ./dataset/val_split/patient029/patient029_frame01.nii
Normalizing: ./dataset/val_split/patient029/patient029_frame12

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame09_gt.nii has unexpected scales in sform

NiftiImageIO (0x55cefe3b8630): ./dataset/val_split/patient098/patient098_frame09_gt.nii has unexpected scales in sform



Resizing: ./dataset/val_split/patient005/patient005_frame01.nii
Resizing: ./dataset/val_split/patient005/patient005_frame01_gt.nii
Resizing: ./dataset/val_split/patient005/patient005_frame13.nii
Resizing: ./dataset/val_split/patient005/patient005_frame13_gt.nii
Resizing: ./dataset/val_split/patient055/patient055_frame01.nii
Resizing: ./dataset/val_split/patient055/patient055_frame01_gt.nii
Resizing: ./dataset/val_split/patient055/patient055_frame10.nii
Resizing: ./dataset/val_split/patient055/patient055_frame10_gt.nii
Resizing: ./dataset/val_split/patient076/patient076_frame01.nii
Resizing: ./dataset/val_split/patient076/patient076_frame01_gt.nii
Resizing: ./dataset/val_split/patient076/patient076_frame12.nii
Resizing: ./dataset/val_split/patient076/patient076_frame12_gt.nii
Resizing: ./dataset/val_split/patient012/patient012_frame01.nii
Resizing: ./dataset/val_split/patient012/patient012_frame01_gt.nii
Resizing: ./dataset/val_split/patient012/patient012_frame13.nii
Resizing: ./dataset